In [1]:
!pip install transformers torch safetensors


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_directory = r'C:\Users\Het\Desktop\Het\Text_classification\trained_model'

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_directory)

# Load the model
model = AutoModelForSequenceClassification.from_pretrained(model_directory, use_safetensors=True)

In [18]:
extracted_text = """
 it is eight years despite filing affidavits in since hc issued a bombay high court assuring series of directions that they will not erect illegal ordering the bmc and hoardings political parties the government to take continue to deface the action against metropolis in contempt of the hoardings judiciary eknath recently on both authorities shindes birthday the city was appear to be swathed in hoardings greeting taking these ravikiran deshmukh the deputy chief minister instructions lightly remili
"""

In [19]:
inputs = tokenizer(extracted_text, truncation=True, padding=True, return_tensors="pt")

In [20]:
import torch

# Ensure the model is in evaluation mode
model.eval()

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=1).item()

# Map the predicted class ID to the corresponding label
labels = ['politics', 'sports', 'business', 'entertainment']  # Adjust based on your model's training
predicted_label = labels[predicted_class_id]

print(f"Predicted Category: {predicted_label}")

Predicted Category: politics


In [1]:
!pip install transformers evaluate torch

  Using cached xxhash-3.5.0-cp311-cp311-win_amd64.whl.metadata (13 kB)
  Using cached multiprocess-0.70.17-py311-none-any.whl.metadata (7.2 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached attrs-25.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached frozenlist-1.5.0-cp311-cp311-win_amd64.whl.metadata (14 kB)
  Using cached multidict-6.1.0-cp311-cp311-win_amd64.whl.metadata (5.1 kB)
  Using cached propcache-0.2.1-cp311-cp311-win_amd64.whl.metadata (9.5 kB)
  Using cached yarl-1.18.3-cp311-cp311-win_amd64.whl.metadata (71 kB)
   ---------------------------------------- 0.0/84.0 kB ? eta -:--:--
   ---------------------------------------  81.9/84.0 kB 4.5 MB/s eta 0:00:01
   ---------------------------------------- 84.0/84.0 kB 670.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/484.9 kB ? eta -:--:--
   -------------- ------------------------- 174.1/484.9 k


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_directory = r'C:\Users\Het\Desktop\Het\Text_classification\trained_model'

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_directory)

# Load the model
model = AutoModelForSequenceClassification.from_pretrained(model_directory)

c:\Users\Het\Desktop\Het\newspaper\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from datasets import Dataset

# Example data
data = {
    'text': [
        "The local sports team secured a remarkable victory in the championship game last night.",
        "The government passed a new law today.",
        "Stock markets are rising amid market optimism.",
        "A new blockbuster movie was released this weekend."
    ],
    'label': [1, 0, 2, 3]  # Example labels: 0=politics, 1=sports, 2=business, 3=entertainment
}

# Create a Dataset
dataset = Dataset.from_dict(data)

# Tokenize the dataset
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 4/4 [00:00<00:00, 31.92 examples/s]


In [4]:
import evaluate
import numpy as np

accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='weighted')
    return {**accuracy, **f1}

In [1]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    per_device_eval_batch_size=16,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_dataset,
    compute_metrics=compute_metrics
)

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

c:\Users\Het\Desktop\Het\newspaper\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'model' is not defined

In [7]:
!pip install --upgrade accelerate


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/342.1 kB ? eta -:--:--
   -------------------- ------------------- 174.1/342.1 kB 3.6 MB/s eta 0:00:01
   ---------------------------------------  337.9/342.1 kB 3.5 MB/s eta 0:00:01
   ---------------------------------------  337.9/342.1 kB 3.5 MB/s eta 0:00:01
   ---------------------------------------  337.9/342.1 kB 3.5 MB/s eta 0:00:01
   ---------------------------------------- 342.1/342.1 kB 1.5 MB/s eta 0:00:00
